# **Data Acquisition II**

## **Ajuste de decimais e encoding**

### **Decimais**

```
df  = pd.read_csv("/content/decimals.txt",
                sep=";", 
                decimal=",", 
                thousands=".", 
                index_col=False)
df
```

```
ANO 	ID_SEG 	COD_UF 	RENDA 	MES 	SB_CONS 	SB_ESCO 	SB_TRAT 	SB_U 	SB_G 	POP 	V10 	V11 	V12 	URG
0 	2012 	4 	10 	19712 	4 	28 	164 	3 	16 	0 	5274 	0.530906 	3.109594 	0.107143 	0.303375
1 	2012 	4 	10 	2547 	5 	13 	0 	6 	23 	0 	5274 	0.246492 	0.000000 	0.461538 	0.436102
2 	2012 	4 	10 	54598 	6 	8 	135 	7 	12 	0 	5274 	0.151688 	2.559727 	0.875000 	0.227531
3 	2012 	4 	10 	20712 	7 	0 	0 	0 	0 	0 	5274 	0.000000 	0.000000 	NaN 	0.000000
```


```df.dtypes```

```
NO          int64
ID_SEG       int64
COD_UF       int64
RENDA        int64
MES          int64
SB_CONS      int64
SB_ESCO      int64
SB_TRAT      int64
SB_U         int64
SB_G         int64
POP          int64
V10        float64
V11        float64
V12        float64
URG        float64
dtype: object
```

### **Encoding**

**Encoding** é uma técnica que define regras para armazenar os code points dos caracteres que compõem as _string_ na memória do computador. Existem vários deles: _UTF-8_, _ISO-8859-1_, _UCS-2_, _windows-1252_ e assim por diante.
```
encoding = 'latin1'
encoding = 'ISO-8859-1'
encoding = 'cp1251'
encoding = 'utf-8'
```